### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M12D20', 'UTC 12:25')


# Settings

In [2]:
MAXTASKS = 100
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,output_url,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,NaN,1513767540018,reduced_global_historical_combined_short_trend...,KNCUA2JFJNEH4CEJQMPQZACD,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767540018
1,NaN,1513767535651,reduced_global_historical_combined_long_trend_...,7O45GAOBJVNAWNJNUISPYT75,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767535651
2,NaN,1513767531305,reduced_global_historical_combined_short_mean_...,FPQILRRJXN4OTOURVE5ZVENA,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767531305
3,NaN,1513767526893,reduced_global_historical_combined_long_mean_W...,OPE5RSZAYPR5I2SPSS25H6FW,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767526893
4,NaN,1513767523019,reduced_global_historical_combined_short_trend...,5Z7LVQML7FHLNOYYEHE66QLR,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767523019
5,NaN,1513767518626,reduced_global_historical_combined_short_trend...,QZJBLOV72BUYM7ZZ5SPFES6L,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767518626
6,NaN,1513767514777,reduced_global_historical_combined_short_trend...,SVOZEC5UGDD2T5LJP5FIT2RG,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767514777
7,NaN,1513767510422,reduced_global_historical_combined_short_trend...,MY5UJLNGI6RRVOZOHDLCZNBU,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767510422
8,NaN,1513767506040,reduced_global_historical_combined_short_trend...,UFBZ3ASPV6LQWSHNLG26SLC3,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767506040
9,NaN,1513767501585,reduced_global_historical_combined_short_trend...,B3GDRHDICL3I2DILXMRTK7VJ,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513767501585


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)